In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
# Loading dataset
dataf = pd.read_csv("https://raw.githubusercontent.com/albanda/ce888/master/lab3/bank-additional-full.csv", delimiter=";")
dataf
features = ["age",	"job",	"marital"	,"education"	,"default",	"housing",	"loan",	"contact"	,"month",	"day_of_week"	,"campaign",	"pdays"	,"previous"	,"poutcome",	"emp.var.rate"	,"cons.price.idx"	,
            "cons.conf.idx",	"euribor3m",	"nr.employed"]


dataf[features].head()
#print("The dataset has %d columns and %d rows" % (dataframe.shape[1], dataframe.shape[0]))

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0


In [3]:
dataf.shape

(41188, 21)

In [4]:
outcome=["y"]
dataf[outcome]

,y
0,no
1,no
2,no
3,no
4,no
...,...
41183,yes
41184,no
41185,no
41186,yes


In [5]:
# Finding no of values column wise
dataf.isna().sum()


age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [6]:
# Droping rows
print("Number of rows before dropping: %d" % len(dataf))
dataf = dataf.dropna()
print("Number of rows after dropping: %d" % len(dataf))

Number of rows before dropping: 41188
Number of rows after dropping: 41188


In [7]:
# Deleting duration
dataf = dataf.drop('duration', 1)

In [8]:
dataf['marital'].unique()

array(['married', 'single', 'divorced', 'unknown'], dtype=object)

In [9]:
y=["job","education","default","month","housing","loan","contact","day_of_week"]
y

['job',
 'education',
 'default',
 'month',
 'housing',
 'loan',
 'contact',
 'day_of_week']

In [10]:

y=["job","education","default","month","housing","loan","contact","day_of_week","y"]
dataf[y] = dataf[y].apply(LabelEncoder().fit_transform)

dataf.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,married,0,0,0,0,1,6,1,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,7,married,3,1,0,0,1,6,1,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,married,3,0,2,0,1,6,1,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,married,1,0,0,0,1,6,1,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,married,3,0,0,2,1,6,1,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


In [11]:
dataf

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,married,0,0,0,0,1,6,1,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,7,married,3,1,0,0,1,6,1,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,married,3,0,2,0,1,6,1,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,married,1,0,0,0,1,6,1,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,married,3,0,0,2,1,6,1,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,5,married,5,0,2,0,0,7,0,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1
41184,46,1,married,5,0,0,0,0,7,0,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41185,56,5,married,6,0,2,0,0,7,0,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41186,44,9,married,5,0,0,0,0,7,0,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1


In [12]:

oneh=["marital","poutcome"]
df_dummies = pd.get_dummies(dataf[oneh])
df_dummies

,marital_divorced,marital_married,marital_single,marital_unknown,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0,1,0,0,0,1,0
1,0,1,0,0,0,1,0
2,0,1,0,0,0,1,0
3,0,1,0,0,0,1,0
4,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...
41183,0,1,0,0,0,1,0
41184,0,1,0,0,0,1,0
41185,0,1,0,0,0,1,0
41186,0,1,0,0,0,1,0


In [13]:
dataf=dataf.join(df_dummies)
dataf


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,euribor3m,nr.employed,y,marital_divorced,marital_married,marital_single,marital_unknown,poutcome_failure,poutcome_nonexistent,poutcome_success
0,56,3,married,0,0,0,0,1,6,1,...,4.857,5191.0,0,0,1,0,0,0,1,0
1,57,7,married,3,1,0,0,1,6,1,...,4.857,5191.0,0,0,1,0,0,0,1,0
2,37,7,married,3,0,2,0,1,6,1,...,4.857,5191.0,0,0,1,0,0,0,1,0
3,40,0,married,1,0,0,0,1,6,1,...,4.857,5191.0,0,0,1,0,0,0,1,0
4,56,7,married,3,0,0,2,1,6,1,...,4.857,5191.0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,5,married,5,0,2,0,0,7,0,...,1.028,4963.6,1,0,1,0,0,0,1,0
41184,46,1,married,5,0,0,0,0,7,0,...,1.028,4963.6,0,0,1,0,0,0,1,0
41185,56,5,married,6,0,2,0,0,7,0,...,1.028,4963.6,0,0,1,0,0,0,1,0
41186,44,9,married,5,0,0,0,0,7,0,...,1.028,4963.6,1,0,1,0,0,0,1,0


In [14]:
dataf=dataf.drop({'marital_unknown','poutcome_nonexistent',"poutcome","marital"},1)
dataf

,age,job,education,default,housing,loan,contact,month,day_of_week,campaign,...,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,marital_divorced,marital_married,marital_single,poutcome_failure,poutcome_success
0,56,3,0,0,0,0,1,6,1,1,...,93.994,-36.4,4.857,5191.0,0,0,1,0,0,0
1,57,7,3,1,0,0,1,6,1,1,...,93.994,-36.4,4.857,5191.0,0,0,1,0,0,0
2,37,7,3,0,2,0,1,6,1,1,...,93.994,-36.4,4.857,5191.0,0,0,1,0,0,0
3,40,0,1,0,0,0,1,6,1,1,...,93.994,-36.4,4.857,5191.0,0,0,1,0,0,0
4,56,7,3,0,0,2,1,6,1,1,...,93.994,-36.4,4.857,5191.0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,5,5,0,2,0,0,7,0,1,...,94.767,-50.8,1.028,4963.6,1,0,1,0,0,0
41184,46,1,5,0,0,0,0,7,0,1,...,94.767,-50.8,1.028,4963.6,0,0,1,0,0,0
41185,56,5,6,0,2,0,0,7,0,2,...,94.767,-50.8,1.028,4963.6,0,0,1,0,0,0
41186,44,9,5,0,0,0,0,7,0,1,...,94.767,-50.8,1.028,4963.6,1,0,1,0,0,0


In [15]:

outcomes = ["y"]
n_bins = 20

X_df = dataf.copy()
y_df = dataf[outcomes].copy()



In [16]:
from sklearn.model_selection import train_test_split
X = X_df.loc[:, X_df.columns != 'y']
y = X_df.loc[:, X_df.columns == 'y']

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=50)

In [18]:
dummy_clf = DummyClassifier(strategy='stratified')
dummy_clf.fit(X_train, y_train)
print("Dummy ACC: %.2f" % acc(y, dummy_clf.predict(X)))

Dummy ACC: 0.80


In [19]:
y_pred=dummy_clf.predict(X_test)

In [20]:
# Confusion matrix and classification report
from sklearn.metrics import classification_report, confusion_matrix,plot_confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[6491  797]
 [ 841  109]]
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      7288
           1       0.12      0.11      0.12       950

    accuracy                           0.80      8238
   macro avg       0.50      0.50      0.50      8238
weighted avg       0.80      0.80      0.80      8238



In [21]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

clf = RandomForestClassifier(n_estimators=400, max_depth=8)

scores = cross_val_score(clf, X_train, y_train, cv=10, scoring=make_scorer(f1_score))  # cross-validation
print("ACC: " ,scores.mean())

C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed whe

ACC:  0.32880362892221404


In [22]:
clf_new = RandomForestClassifier(n_estimators=300, max_depth=6)

scores1 = cross_val_score(clf_new, X_train, y_train, cv=10, scoring=make_scorer(f1_score))  # cross-validation
print("ACC: " ,scores1.mean())

C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed whe

ACC:  0.2993642163809861


In [24]:
clf_new2 = RandomForestClassifier(n_estimators=100, max_depth=12,random_state=60)
a=clf_new2.fit(X_train, y_train)
clf_new2.score(X_train, y_train)

<ipython-input-24-f3a8387cbec7>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  a=clf_new2.fit(X_train, y_train)


0.9301365705614567

In [25]:
y_pred=clf_new2.predict(X_test)

In [26]:
# Confusion matrix and classification report
from sklearn.metrics import classification_report, confusion_matrix,plot_confusion_matrix,f1_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_test, y_pred))

[[7139  149]
 [ 706  244]]
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      7288
           1       0.62      0.26      0.36       950

    accuracy                           0.90      8238
   macro avg       0.77      0.62      0.65      8238
weighted avg       0.88      0.90      0.88      8238

0.3633655994043187


In [27]:
clf3 = RandomForestClassifier(n_estimators=1000, max_depth=10,random_state=60)
clf3.fit(X_train,y_train)
importances = clf3.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf3.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
print(indices)

# Print the feature ranking
print("Feature ranking:")



<ipython-input-27-85c888189b0c>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf3.fit(X_train,y_train)


[15 16 10 21 14  0 12 13  7  8  1  9  2 11  6 20  4  5 19 18  3 17]
Feature ranking:
